# RNNLM

*Recurrent Neural Network Language Model*

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchtext import transforms
from torchtext.vocab import build_vocab_from_iterator
from torchvision.transforms import Compose
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import List


---

## Dataset

In [ ]:
df = pd.read_excel('data/JEC_basic_sentence_v1-3.xls', header=None)
df.columns = ['id', 'japanese', 'english', 'chinese']
print('num of data: ', len(df))
df.head()

num of data:  5304


,id,japanese,english,chinese
0,#0001,Xではないかとつくづく疑問に思う,I often wonder if it might be X.,难道不会是X吗，我实在是感到怀疑。
1,#0002,Xがいいなといつも思います,I always think X would be nice.,我总觉得X不错。
2,#0003,それがあるようにいつも思います,It always seems like it is there.,我总觉得那好像是有的。
3,#0004,それが多すぎないかと正直思う,I honestly feel like there is too much.,老实说我觉得那太多了。
4,#0005,山田はみんなに好かれるタイプの人だと思う,I think that Yamada is the type everybody likes.,我想山田是受大家欢迎的那种人。


In [ ]:
nlp_en = spacy.load('en_core_web_sm')
nlp_ja = spacy.load('ja_core_news_sm')
def tokenize(data: List[str], l='en') -> List[List[str]]:
    nlp = eval('nlp_' + l)
    return [[token.text for token in nlp(sentence)] for sentence in data]

In [ ]:
text_ja = tokenize(df['japanese'], l='ja')
text_en = tokenize(df['english'], l='en')

In [ ]:
text_en

[['I', 'often', 'wonder', 'if', 'it', 'might', 'be', 'X.'],
 ['I', 'always', 'think', 'X', 'would', 'be', 'nice', '.'],
 ['It', 'always', 'seems', 'like', 'it', 'is', 'there', '.'],
 ['I', 'honestly', 'feel', 'like', 'there', 'is', 'too', 'much', '.'],
 ['I',
  'think',
  'that',
  'Yamada',
  'is',
  'the',
  'type',
  'everybody',
  'likes',
  '.'],
 ['Someone', 'thought', 'that', '〜'],
 ['X', 'seems', 'like', 'it', "'s", 'really', 'tough', '.'],
 ['I', 'think', 'X', 'is', 'just', 'a', 'matter', 'of', 'time', '.'],
 ['I',
  'think',
  'that',
  'X',
  'will',
  'become',
  'an',
  'issue',
  'in',
  'the',
  'future',
  '.'],
 ['I',
  'think',
  'this',
  'was',
  'made',
  'around',
  'the',
  'Momoyama',
  'Period',
  '.'],
 ['I',
  'think',
  'this',
  'movie',
  'is',
  'from',
  'the',
  'early',
  'Showa',
  'Period',
  '.'],
 ['Who', 'thinks', 'they', "'re", 'going', 'to', 'win', '?'],
 ['I', 'wonder', 'about', 'X.'],
 ['I', 'do', "n't", 'think', 'he', 'was', 'able', 'do', 'it

#### 前処理

In [ ]:
len(max(text_ja, key=len))

27

In [ ]:
pad, bos, eos, unk = '<pad>', '<bos>', '<eos>', '<unk>'
max_len = 30
specials = [pad, bos, eos, unk]
vocab_ja = build_vocab_from_iterator(text_ja, specials=specials)
vocab_en = build_vocab_from_iterator(text_en, specials=specials)

transform_ja = Compose([
    transforms.Truncate(max_len),
    transforms.AddToken(bos, begin=True),
    transforms.AddToken(eos, begin=False),
    transforms.VocabTransform(vocab_ja),
    transforms.ToTensor(),
    transforms.PadTransform(max_len, vocab_ja[pad])
])

transform_en = Compose([
    transforms.Truncate(max_len),
    transforms.AddToken(bos, begin=True),
    transforms.AddToken(eos, begin=False),
    transforms.VocabTransform(vocab_en),
    transforms.ToTensor(),
    transforms.PadTransform(max_len, vocab_en[pad])
])

In [ ]:
ex2 =text_ja[2]
ex2

['それ', 'が', 'ある', 'よう', 'に', 'いつも', '思い', 'ます']

In [ ]:
transform_ja(ex2)

tensor([  1,  20,   4, 253, 104,   7,  88, 161,   9,   2,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0])

##### Vocab

数値化

In [ ]:
v = build_vocab_from_iterator(ex2, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

In [ ]:
v.get_stoi()

{'よ': 14,
 '<unk>': 0,
 '<pad>': 1,
 'あ': 5,
 '<eos>': 3,
 'す': 8,
 '<bos>': 2,
 'に': 11,
 'い': 4,
 'う': 6,
 'れ': 16,
 'る': 15,
 'も': 13,
 'が': 7,
 '思': 17,
 'そ': 9,
 'つ': 10,
 'ま': 12}

In [ ]:
t = transforms.VocabTransform(v)
tokens = t(ex2)
tokens

RuntimeError: Token それ not found and default index is not set
Exception raised from __getitem__ at /__w/text/text/pytorch/text/torchtext/csrc/vocab.cpp:43 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7f57704784d7 in /home/komiya/.pyenv/versions/miniforge3-4.10.3-10/envs/nlp/lib/python3.11/site-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x64 (0x7f577044236b in /home/komiya/.pyenv/versions/miniforge3-4.10.3-10/envs/nlp/lib/python3.11/site-packages/torch/lib/libc10.so)
frame #2: torchtext::Vocab::__getitem__(c10::basic_string_view<char> const&) const + 0x384 (0x7f5676f0dde4 in /home/komiya/.pyenv/versions/miniforge3-4.10.3-10/envs/nlp/lib/python3.11/site-packages/torchtext/lib/libtorchtext.so)
frame #3: <unknown function> + 0x1e5a0 (0x7f5676df95a0 in /home/komiya/.pyenv/versions/miniforge3-4.10.3-10/envs/nlp/lib/python3.11/site-packages/torchtext/_tor

##### Padding

長さの調整

In [ ]:
ex1

['I', 'often', 'wonder', 'if', 'it', 'might', 'be', 'X.']

In [ ]:
t = transforms.PadTransform(10, 0)
tokens = torch.tensor(tokens)
t(tokens)

tensor([ 4, 14, 16, 10, 11, 12,  5,  8,  0,  0])

Dataset

In [ ]:
class TextDataset(Dataset):
    def __init__(self, in_text, out_text, in_transform, out_transform):
        self.in_text = in_text
        self.out_text = out_text
        self.in_transform = in_transform
        self.out_transform = out_transform
        self.n_samples = len(in_text)

    def __getitem__(self, index):
        in_text = self.in_text[index]
        out_text = self.out_text[index]
        in_text = self.in_transform(in_text)
        out_text = self.out_transform(out_text)
        return in_text, out_text

    def __len__(self):
        return self.n_samples

In [ ]:
dataset = TextDataset(text_ja, text_en, transform_ja, transform_en)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

### Train

In [ ]:
def train(model, optimizer, n_epochs):
    model.train()
    for epoch in range(1, n_epochs + 1):
        for x, y in tqdm(dataloader, desc=f'Epoch {epoch}/{n_epochs}'):
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
        print('loss:', loss.item(), flush=True)